## 1. Importação e instalação das dependê

In [ ]:
#pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe==0.8.6.2 sklearn matplotlib --user

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

## 2. Mapeamento dos pontos-chave utilizando MediaPipe Holistic

In [2]:
mp_holistic = mp.solutions.holistic # Modelo holístico
mp_drawing = mp.solutions.drawing_utils # Desenho dos pontos

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # CONVERSÃO DAS CORES BGR para RGB
    image.flags.writeable = False                  # Imagem não é mais alterável
    results = model.process(image)                 # Faz a predição
    image.flags.writeable = True                   # Imagem volta a ser alterável 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # CONVERSÃO DAS CORES RGB para BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS) # Desenha conexões do rosto
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Desenha conexões da mão direita

In [5]:
def draw_styled_landmarks(image, results):
    # Desenha conexões do rosto
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Desenha conexões da pose
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão esquerda
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Desenha conexões da mão direita  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

## 3. Extrair os ponto-chave

In [6]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

## 4. Configuração das pastas para dados

In [7]:
# Caminho para exportar o dado dos gestos
DATA_PATH = os.path.join('MP_Data') 

# Acoes que iremos detectar
actions = np.array(['ola','boa noite','eu_te_amo','lingua_de_sinais', 'surdo', 'conhecer', 'amigo','entender','denovo','estudar',])

# 65 videos para cada pose
no_sequences = 50

# Cada possuira 30 frames
sequence_length = 30

In [8]:
for action in actions: # para cada gesto
    for sequence in range(no_sequences): # para cada frame
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence))) # cria o diretório
        except:
            pass

## 5. Coleta os ponto-chave para treinamento e testes

In [ ]:
cap = cv2.VideoCapture(0)
# Configura o modelo mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop pelos gestos
    for action in actions:
        # Loop pelas sequências (videos)
        for sequence in range(no_sequences):
            # Loop pelos frames
            for frame_num in range(sequence_length):

                # Le a camera
                ret, frame = cap.read()

                # Faz as detecções
                image, results = mediapipe_detection(frame, holistic)
                # print(results)

                # Desenha os ponto-chave
                draw_styled_landmarks(image, results)
                
                # se for o início de um vídeo
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostra na tela
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Mostra na tela
                    cv2.imshow('OpenCV Feed', image)
                
                
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Sai do programa
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

## 6. Configuração e treinamento da Rede Neural LSTM

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
# Coleta os dados das pastas
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [13]:
X = np.array(sequences) # dados de treinamento

In [14]:
y = to_categorical(labels).astype(int) # rótulo dos sinais

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1) # separação dos dados de treinamento, 20% será utilizado para testes

In [16]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [17]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [18]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [19]:
#from tensorflow.keras.callbacks import EarlyStopping
#early_stop = EarlyStopping(monitor='loss',mode='min',verbose=1,patience=25)
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
15/15 [==============================] - 5s 121ms/step - loss: 2.7005 - categorical_accuracy: 0.1096
Epoch 2/1000
15/15 [==============================] - 1s 98ms/step - loss: 2.4205 - categorical_accuracy: 0.1603
Epoch 3/1000
15/15 [==============================] - 1s 87ms/step - loss: 2.2032 - categorical_accuracy: 0.1493
Epoch 4/1000
15/15 [==============================] - 1s 86ms/step - loss: 2.1245 - categorical_accuracy: 0.1887
Epoch 5/1000
15/15 [==============================] - 1s 85ms/step - loss: 1.9420 - categorical_accuracy: 0.2445
Epoch 6/1000
15/15 [==============================] - 1s 88ms/step - loss: 1.7974 - categorical_accuracy: 0.3494
Epoch 7/1000
15/15 [==============================] - 2s 102ms/step - loss: 2.3776 - categorical_accuracy: 0.2506
Epoch 8/1000
15/15 [==============================] - 1s 98ms/step - loss: 1.4928 - categorical_accuracy: 0.3626 0s - loss: 1.5014 - categorical_accuracy: 
Epoch 9/1000
15/15 [==============================]

15/15 [==============================] - 1s 87ms/step - loss: 0.6327 - categorical_accuracy: 0.7264 0s - loss: 0.6149 - categorical_accuracy
Epoch 71/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.5636 - categorical_accuracy: 0.7888 1s - loss: 0.4810 - ca
Epoch 72/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.5209 - categorical_accuracy: 0.8104
Epoch 73/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.3898 - categorical_accuracy: 0.8659
Epoch 74/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.4668 - categorical_accuracy: 0.8385
Epoch 75/1000
15/15 [==============================] - 1s 96ms/step - loss: 0.4344 - categorical_accuracy: 0.8446
Epoch 76/1000
15/15 [==============================] - 1s 89ms/step - loss: 4.2865 - categorical_accuracy: 0.6480 0s - loss: 5.0201 - categorical_accu
Epoch 77/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.7997 - categorical_accuracy: 0.6494 0

15/15 [==============================] - 1s 90ms/step - loss: 0.1554 - categorical_accuracy: 0.9387
Epoch 141/1000
15/15 [==============================] - 1s 89ms/step - loss: 0.2373 - categorical_accuracy: 0.9212
Epoch 142/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.1744 - categorical_accuracy: 0.9285
Epoch 143/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.1428 - categorical_accuracy: 0.9577
Epoch 144/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.1667 - categorical_accuracy: 0.9345
Epoch 145/1000
15/15 [==============================] - 1s 88ms/step - loss: 0.1324 - categorical_accuracy: 0.9429
Epoch 146/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.1439 - categorical_accuracy: 0.9448
Epoch 147/1000
15/15 [==============================] - 1s 92ms/step - loss: 0.1463 - categorical_accuracy: 0.9476
Epoch 148/1000
15/15 [==============================] - 1s 88ms/step - loss: 0.7120 - categoric

15/15 [==============================] - 1s 87ms/step - loss: 0.5276 - categorical_accuracy: 0.7804
Epoch 210/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.4483 - categorical_accuracy: 0.8129
Epoch 211/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.4429 - categorical_accuracy: 0.8132
Epoch 212/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.3934 - categorical_accuracy: 0.8511
Epoch 213/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.4199 - categorical_accuracy: 0.8451
Epoch 214/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.3457 - categorical_accuracy: 0.8717
Epoch 215/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.3606 - categorical_accuracy: 0.8739
Epoch 216/1000
15/15 [==============================] - 1s 90ms/step - loss: 0.9228 - categorical_accuracy: 0.6835
Epoch 217/1000
15/15 [==============================] - 1s 93ms/step - loss: 1.0534 - categoric

15/15 [==============================] - 1s 96ms/step - loss: 0.2039 - categorical_accuracy: 0.9316
Epoch 280/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.1899 - categorical_accuracy: 0.9385
Epoch 281/1000
15/15 [==============================] - 1s 93ms/step - loss: 0.1499 - categorical_accuracy: 0.9473
Epoch 282/1000
15/15 [==============================] - 1s 100ms/step - loss: 0.1212 - categorical_accuracy: 0.9580
Epoch 283/1000
15/15 [==============================] - 2s 110ms/step - loss: 0.1320 - categorical_accuracy: 0.9521
Epoch 284/1000
15/15 [==============================] - 2s 105ms/step - loss: 0.0965 - categorical_accuracy: 0.9779
Epoch 285/1000
15/15 [==============================] - 1s 95ms/step - loss: 0.0630 - categorical_accuracy: 0.9866
Epoch 286/1000
15/15 [==============================] - 2s 109ms/step - loss: 0.1087 - categorical_accuracy: 0.9554
Epoch 287/1000
15/15 [==============================] - 1s 100ms/step - loss: 0.0722 - cate

...

15/15 [==============================] - 1s 86ms/step - loss: 0.1084 - categorical_accuracy: 0.9715
Epoch 696/1000
15/15 [==============================] - 1s 87ms/step - loss: 2.8504 - categorical_accuracy: 0.5564
Epoch 697/1000
15/15 [==============================] - 1s 84ms/step - loss: 1.1257 - categorical_accuracy: 0.6250
Epoch 698/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.4720 - categorical_accuracy: 0.8568 0s - loss: 0.4837 - categorical_accuracy: 
Epoch 699/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.2762 - categorical_accuracy: 0.9175
Epoch 700/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.1251 - categorical_accuracy: 0.9730
Epoch 701/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.2545 - categorical_accuracy: 0.9353
Epoch 702/1000
15/15 [==============================] - 1s 87ms/step - loss: 0.2497 - categorical_accuracy: 0.9580
Epoch 703/1000
15/15 [==============================

15/15 [==============================] - 1s 88ms/step - loss: 1.1413 - categorical_accuracy: 0.5438
Epoch 764/1000
15/15 [==============================] - 1s 84ms/step - loss: 1.2213 - categorical_accuracy: 0.5133
Epoch 765/1000
15/15 [==============================] - 1s 84ms/step - loss: 1.0542 - categorical_accuracy: 0.5653
Epoch 766/1000
15/15 [==============================] - 1s 90ms/step - loss: 1.0926 - categorical_accuracy: 0.5307
Epoch 767/1000
15/15 [==============================] - 1s 89ms/step - loss: 1.2111 - categorical_accuracy: 0.5405
Epoch 768/1000
15/15 [==============================] - 1s 89ms/step - loss: 0.9595 - categorical_accuracy: 0.6080
Epoch 769/1000
15/15 [==============================] - 1s 89ms/step - loss: 1.5152 - categorical_accuracy: 0.4323
Epoch 770/1000
15/15 [==============================] - 1s 86ms/step - loss: 1.1055 - categorical_accuracy: 0.5626
Epoch 771/1000
15/15 [==============================] - 1s 85ms/step - loss: 1.0104 - categoric

15/15 [==============================] - 1s 85ms/step - loss: 0.7743 - categorical_accuracy: 0.6999
Epoch 834/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.6087 - categorical_accuracy: 0.7322
Epoch 835/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.4791 - categorical_accuracy: 0.8295
Epoch 836/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.3954 - categorical_accuracy: 0.8494
Epoch 837/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.4194 - categorical_accuracy: 0.8371
Epoch 838/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.3599 - categorical_accuracy: 0.8651
Epoch 839/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.3481 - categorical_accuracy: 0.8855
Epoch 840/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.4251 - categorical_accuracy: 0.8255
Epoch 841/1000
15/15 [==============================] - 1s 86ms/step - loss: 0.4138 - categoric

15/15 [==============================] - 1s 84ms/step - loss: 0.2099 - categorical_accuracy: 0.9201
Epoch 904/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.3320 - categorical_accuracy: 0.8717
Epoch 905/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.2039 - categorical_accuracy: 0.9179
Epoch 906/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.2374 - categorical_accuracy: 0.9068
Epoch 907/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1701 - categorical_accuracy: 0.9376
Epoch 908/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1719 - categorical_accuracy: 0.9313
Epoch 909/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.3473 - categorical_accuracy: 0.8800
Epoch 910/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.6280 - categorical_accuracy: 0.7767
Epoch 911/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.3367 - categoric

15/15 [==============================] - 1s 83ms/step - loss: 0.1039 - categorical_accuracy: 0.9714
Epoch 975/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1146 - categorical_accuracy: 0.9613
Epoch 976/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.1087 - categorical_accuracy: 0.9627
Epoch 977/1000
15/15 [==============================] - 1s 82ms/step - loss: 0.1718 - categorical_accuracy: 0.9339
Epoch 978/1000
15/15 [==============================] - 1s 82ms/step - loss: 0.2326 - categorical_accuracy: 0.9169
Epoch 979/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.3722 - categorical_accuracy: 0.8288
Epoch 980/1000
15/15 [==============================] - 1s 85ms/step - loss: 0.2154 - categorical_accuracy: 0.9354
Epoch 981/1000
15/15 [==============================] - 1s 83ms/step - loss: 0.1789 - categorical_accuracy: 0.9398
Epoch 982/1000
15/15 [==============================] - 1s 84ms/step - loss: 0.2318 - categoric

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 64)            442112    
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 128)           98816     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 596,906
Trainable params: 596,906
Non-trainable params: 0
__________________________________________________

## 7. Salva o modelo

In [21]:
model.save('action10_v5.h5') # salva o modelo

In [ ]:
model.load_weights('action10_v4.h5') # carrega o modelo

In [ ]:
model.evaluate(X_test,y_test) # teste da acurácia

## 8. Teste em tempo real

In [22]:
from scipy import stats

In [23]:
# função que exibe a porcentagem de cada gesto
colors = [(245,117,16), (117,245,16), (16,117,245),(255, 191, 0),(255, 255, 0),(191, 255, 0),(128, 255, 0),(0, 255, 191),(128, 0, 255),(255, 0, 64)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [24]:
sequence = []
sentence = []
predictions = []
threshold = 0.5 # tempo de espera para confirmar o sinal

cap = cv2.VideoCapture(0)
# configura o modelo mediapipe
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # le a camera
        ret, frame = cap.read()

        # faz as detecções
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # desenha os ponto-chave
        draw_styled_landmarks(image, results)
        
        # lógica da predição
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
            # lógica da visualização
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # vizualicação das probabilidades
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # mostra na tela
        cv2.imshow('OpenCV Feed', image)

        # sai do programa
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'me

<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
amigo
<class 'mediapipe.python.solution_base.SolutionOutputs'

<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.python.solution_base.SolutionOutputs'>
conhecer
<class 'mediapipe.p

<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lingua_de_sinais
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lingua_de_sinais
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lingua_de_sinais
<class 'mediapipe.python.solution_base.SolutionOutputs'>
lingua_de_sinais
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs'>
estudar
<class 'mediapipe.python.solution_base.SolutionOutputs

...

eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
eu_te_amo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>


<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
denovo
<class 'mediapipe.python.solution_base.SolutionOutputs'>
boa noite
<class 'mediapipe.python.solution_base.SolutionOu

ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
ola
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.pyt

<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'mediapipe.python.solution_base.SolutionOutputs'>
entender
<class 'me

KeyboardInterrupt: 